# Retrieval Augmented Generation (RAG)

## GPT Definition:

Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval with generative AI models to improve the accuracy and reliability of responses. Instead of relying only on the model’s pre-trained knowledge, RAG retrieves relevant documents or passages from an external knowledge base (such as a database or search index) and feeds them into the model as context. This way, the model can generate answers grounded in up-to-date or domain-specific information, reducing hallucinations and making outputs more factual and context-aware.

## Basic RAG Pipeline  

### **Preprocessing (Building the Knowledge Base):**  
1. Collect a set of relevant documents.  
2. Split documents into smaller, semantically meaningful **chunks** (e.g., paragraphs or sections).  
3. Choose an appropriate **embedding model**.  
4. Convert each chunk into a vector representation (**embedding**).  
5. Store the **(embedding, text-chunk)** pairs for future usage.  

### **Retrieving & Generating (Answering Queries):**  
1. Receive a user query.  
2. Embed the query using the same embedding model.  
3. Perform a **similarity search** between the query embedding and the stored chunk embeddings.  
4. Select the most relevant chunks based on similarity scores.  
5. Construct a prompt by inserting these retrieved chunks alongside the query.  
6. Send the prompt to a **Large Language Model (LLM)** to generate the final response.

# Basic and necessary imports and definitions

In [1]:
import os
import requests
import random
from tqdm import tqdm
import re
import torch
from dotenv import load_dotenv

load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

LLM_MODEL_ID = "google/gemma-2-2b-it" # https://ai.google.dev/gemma/docs/core/model_card_2 https://huggingface.co/google/gemma-2-2b-it
EMBEDDING_MODEL = "BAAI/bge-m3" # https://bge-model.com/bge/bge_m3.html https://huggingface.co/BAAI/bge-m3

FILE_PATH = os.path.join("data", "manual-safebank.pdf")
EMBEDDING_PATH = os.path.join("data", "embeddings.csv")

# Select the most appropriate device for model inference
# Options: "cpu", "cuda" (NVIDIA GPUs), "mps" (Apple Silicon GPUs)
DEVICE = "cpu"
DEVICE = "mps" if torch.backends.mps.is_available() else DEVICE
DEVICE = "cuda" if torch.cuda.is_available() else DEVICE

print(f"Using device: {DEVICE}")

Using device: mps


# Preprocessing

## Loading PDF

For simplicity, only **PDF files** will be considered.

In [2]:
assert os.path.exists(FILE_PATH), f"File '{FILE_PATH}' does not exist."

In [3]:
import pymupdf # https://pymupdf.readthedocs.io/en/latest/the-basics.html

def text_formatter(text: str) -> str:
    """"
    Simple text formatter to remove new lines and extra spaces.
    """
    return text.replace("\n", " ").strip()

def load_pdfs_files(file_path: str) -> list:
    pages = list()

    if file_path.endswith(".pdf"): # Process only PDF files
        doc = pymupdf.open(file_path)
        for page in doc: # Loop through each page in the PDF
            text = text_formatter(page.get_text())
            _dict = {
                "page_char_count": len(text),
                "page_word_count": len(text.split(" ")),
                "page_token_count": len(text) / 4,  # Rough estimate -> 1 token ~= 4 chars
                "text": text,
            }
            pages.append(_dict)
    else:
        print(f"Skipping non-PDF file: '{file_path}'")
        return None
            
    return pages

In [4]:
pages = load_pdfs_files(FILE_PATH)
print(f"Total pages extracted: {len(pages)}")

pages[0]

Total pages extracted: 11


{'page_char_count': 1668,
 'page_word_count': 297,
 'page_token_count': 417.0,
 'text': 'Manual de Atendimento e Uso – SafeBank    1. Introdução  Objetivo do Manual   Este manual tem como objetivo orientar clientes e atendentes sobre o funcionamento  completo do banco digital, abordando desde o acesso à conta, funcionalidades e serviços,  até procedimentos de segurança, suporte e resolução de problemas. O documento também  serve como material de consulta para dúvidas frequentes.  Visão geral do banco digital   Nosso banco digital oferece serviços bancários completos de forma 100% online, por meio  do aplicativo e do site. Entre os principais serviços estão conta digital gratuita,  transferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e  muito mais, sempre com foco em praticidade, segurança e transparência.  Público-alvo   O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções  financeiras modernas, acessíveis e com ges

In [5]:
import pandas as pd

df = pd.DataFrame(pages)
df.describe().round(1)

,page_char_count,page_word_count,page_token_count
count,11.0,11.0,11.0
mean,1759.1,302.1,439.8
std,216.4,35.9,54.1
min,1427.0,242.0,356.8
25%,1621.0,275.0,405.2
50%,1689.0,307.0,422.2
75%,1918.0,328.0,479.5
max,2120.0,357.0,530.0


## Text splitting

Texts will be split into sentences and filtered to remove chunks shorter than a minimum token length threshold.

In [6]:
from spacy.lang.pt import Portuguese

nlp = Portuguese()
nlp.add_pipe("sentencizer")

doc = nlp("Primeira frase. Outra frase?! Terceirta frase!")
print(f"Number of sentences: {len(list(doc.sents))}")
for sent in doc.sents:
    print(f"- {sent.text}")

Number of sentences: 3
- Primeira frase.
- Outra frase?!
- Terceirta frase!


In [7]:
for pag in pages:
    doc = nlp(pag["text"])
    pag["sentences"] = [str(sent.text).strip() for sent in doc.sents]
    pag["sentence_count"] = len(pag["sentences"])

pages[0]

{'page_char_count': 1668,
 'page_word_count': 297,
 'page_token_count': 417.0,
 'text': 'Manual de Atendimento e Uso – SafeBank    1. Introdução  Objetivo do Manual   Este manual tem como objetivo orientar clientes e atendentes sobre o funcionamento  completo do banco digital, abordando desde o acesso à conta, funcionalidades e serviços,  até procedimentos de segurança, suporte e resolução de problemas. O documento também  serve como material de consulta para dúvidas frequentes.  Visão geral do banco digital   Nosso banco digital oferece serviços bancários completos de forma 100% online, por meio  do aplicativo e do site. Entre os principais serviços estão conta digital gratuita,  transferências via Pix, pagamento de boletos, emissão de cartões, investimentos, seguros e  muito mais, sempre com foco em praticidade, segurança e transparência.  Público-alvo   O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções  financeiras modernas, acessíveis e com ges

In [8]:
import pandas as pd

df = pd.DataFrame(pages)
df.describe().round(1)

,page_char_count,page_word_count,page_token_count,sentence_count
count,11.0,11.0,11.0,11.0
mean,1759.1,302.1,439.8,17.0
std,216.4,35.9,54.1,4.6
min,1427.0,242.0,356.8,9.0
25%,1621.0,275.0,405.2,13.5
50%,1689.0,307.0,422.2,19.0
75%,1918.0,328.0,479.5,20.5
max,2120.0,357.0,530.0,23.0


In [9]:
def split_into_chunks(sentences: list[str], num_sentences_per_chunk: int, sentences_overlapping: int) -> list[list[str]]:
    return [sentences[i:i + num_sentences_per_chunk + sentences_overlapping] for i in range(0, len(sentences), num_sentences_per_chunk)]

In [10]:
for pag in pages:
    pag["chunks"] = split_into_chunks(pag["sentences"], num_sentences_per_chunk=5, sentences_overlapping=2)
    pag["chunk_count"] = len(pag["chunks"])

random.sample(pages, 1)[0]["chunks"]

[['Recursos exclusivos do app\u200b  Alguns recursos estão disponíveis apenas no app, como saque por QR Code, notificações  push em tempo real, login por biometria e autenticação em dois fatores via aplicativo.',
  'Permissões e notificações\u200b  O app pode solicitar permissões como localização (para encontrar caixas) e acesso à  câmera (para leitura de QR Code).',
  'Notificações podem ser gerenciadas nas configurações  do app e do sistema operacional.',
  'Erros comuns e soluções\u200b  Erros como “Falha na conexão” ou “Dados inválidos” costumam estar relacionados a  instabilidade de internet ou versões desatualizadas.',
  'Recomenda-se verificar a conexão,  atualizar o app e reiniciar o dispositivo.',
  '11.',
  'Segurança da Conta  Autenticação em dois fatores (2FA)\u200b  O banco digital oferece autenticação em dois fatores para proteger o acesso.'],
 ['11.',
  'Segurança da Conta  Autenticação em dois fatores (2FA)\u200b  O banco digital oferece autenticação em dois fatores par

In [11]:
chunks = list()

for pag in pages:
    for chunk in pag["chunks"]:
        joined_text = " ".join(chunk).replace("  ", " ").strip() # Join sentences in the chunk
        joined_text = re.sub(r'\.([A-Z])', r'. \1', joined_text) # ".A" -> ". A"

        chunks.append({
            "text": joined_text,
            "chunk_char_count": len(joined_text),
            "chunk_word_count": len(joined_text.split(" ")),
            "chunk_token_count": len(joined_text) / 4
        })

random.sample(chunks, 1)[0]

{'text': 'cai no mesmo dia, enquanto DOC pode levar até o dia útil seguinte. Já o Pix funciona 24h por dia, todos os dias da semana, com compensação instantânea. Cadastramento de chaves Pix\u200b O usuário pode cadastrar suas chaves Pix (CPF, CNPJ, e-mail, número de celular ou chave aleatória) diretamente pelo app ou site. Basta acessar a área "Pix", clicar em “Minhas chaves” e seguir o passo a passo para registro, exclusão ou portabilidade. Transferência via QR Code\u200b Transferências Pix podem ser feitas escaneando um QR Code gerado por outra pessoa ou estabelecimento. No app, basta acessar “Pix”, escolher “Pagar com QR Code” e apontar a câmera. O sistema preencherá automaticamente os dados da transação.',
 'chunk_char_count': 697,
 'chunk_word_count': 114,
 'chunk_token_count': 174.25}

In [12]:
df = pd.DataFrame(chunks)
df.describe().round(1)

,chunk_char_count,chunk_word_count,chunk_token_count
count,42.0,42.0,42.0
mean,568.0,88.3,142.0
std,223.9,35.2,56.0
min,70.0,11.0,17.5
25%,491.2,76.8,122.8
50%,580.0,89.5,145.0
75%,659.0,107.2,164.8
max,997.0,161.0,249.2


In [13]:
MIN_TOKENS_LENGTH = 25

print(f"Chunks with less than {MIN_TOKENS_LENGTH} tokens: {sum(df["chunk_token_count"] < MIN_TOKENS_LENGTH)}")

Chunks with less than 25 tokens: 3


In [14]:
def chunks_fitter(text_and_chunks, min_tokens_length=25):
    """
    Join small chunks with the smaller one (previous or next)
    """
    
    new_text_and_chunks = list()
    i = 0
    while i < len(text_and_chunks):
        current_chunk = text_and_chunks[i]
        if current_chunk["chunk_token_count"] < min_tokens_length:
            
            if i == 0: # first chunk, only merge with next
                next_chunk = text_and_chunks[i + 1]
                current_chunk["text"] += " " + next_chunk["text"]
                current_chunk["chunk_char_count"] = len(current_chunk["text"])
                current_chunk["chunk_word_count"] = len(current_chunk["text"].split(" "))
                current_chunk["chunk_token_count"] = len(current_chunk["text"]) / 4
                new_text_and_chunks.append(current_chunk)
                i += 2 # skip the next chunk as it's already merged
            
            elif i == len(text_and_chunks) - 1: # last chunk, only merge with previous
                prev_chunk = new_text_and_chunks[i - 1]
                prev_chunk["text"] += " " + current_chunk["text"]
                prev_chunk["chunk_char_count"] = len(prev_chunk["text"])
                prev_chunk["chunk_word_count"] = len(prev_chunk["text"].split(" "))
                prev_chunk["chunk_token_count"] = len(prev_chunk["text"]) / 4
                i += 1
            
            else: # middle chunk, merge with the smaller of previous or next
                prev_chunk = new_text_and_chunks[-1]
                next_chunk = text_and_chunks[i + 1]
                if prev_chunk["chunk_token_count"] <= next_chunk["chunk_token_count"]:
                    prev_chunk["text"] += " " + current_chunk["text"]
                    prev_chunk["chunk_char_count"] = len(prev_chunk["text"])
                    prev_chunk["chunk_word_count"] = len(prev_chunk["text"].split(" "))
                    prev_chunk["chunk_token_count"] = len(prev_chunk["text"]) / 4
                    i += 1
                else:
                    current_chunk["text"] += " " + next_chunk["text"]
                    current_chunk["chunk_char_count"] = len(current_chunk["text"])
                    current_chunk["chunk_word_count"] = len(current_chunk["text"].split(" "))
                    current_chunk["chunk_token_count"] = len(current_chunk["text"]) / 4
                    new_text_and_chunks.append(current_chunk)
                    i += 2 # skip the next chunk as it's already merged
        else:
            new_text_and_chunks.append(current_chunk)
            i += 1
    return new_text_and_chunks

In [15]:
filttered_chunks = chunks_fitter(chunks, min_tokens_length=MIN_TOKENS_LENGTH)
df = pd.DataFrame(filttered_chunks)
df.describe().round(1)

,chunk_char_count,chunk_word_count,chunk_token_count
count,39.0,39.0,39.0
mean,611.7,95.1,152.9
std,186.0,29.5,46.5
min,102.0,16.0,25.5
25%,543.0,84.5,135.8
50%,590.0,94.0,147.5
75%,709.0,110.0,177.2
max,997.0,161.0,249.2


In [16]:
print(f"Chunks with less than {MIN_TOKENS_LENGTH} tokens: {sum(df["chunk_token_count"] < MIN_TOKENS_LENGTH)}")

Chunks with less than 25 tokens: 0


## Text embedding

In [17]:
from sentence_transformers import SentenceTransformer # https://huggingface.co/sentence-transformers

embedding_model = SentenceTransformer(model_name_or_path=EMBEDDING_MODEL,
                            device=DEVICE,
                            config_kwargs={"use_fp16": True}) # Setting use_fp16 to True speeds up computation with a slight performance degradation

/Users/joaoloss/Documents/ML/LLM_studies/udemy_course/no_frameworks/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
test_sentences = [
    "Olá, como posso ajudar você hoje?",
    "Qual é o status do meu pedido?",
    "Preciso de assistência com minha conta."
]

test_embeddings = embedding_model.encode(sentences=test_sentences,
                                         dtype=torch.float32,
                                         convert_to_tensor=True)
test_embeddings

tensor([[-0.0392,  0.0595, -0.0200,  ..., -0.0197, -0.0414, -0.0247],
        [-0.0268,  0.0013, -0.0271,  ..., -0.0639,  0.0011, -0.0358],
        [-0.0028,  0.0202, -0.0625,  ..., -0.0128, -0.0418, -0.0130]],
       device='mps:0')

In [19]:
print(f"Embedding dim: {test_embeddings[0].shape[0]}")

Embedding dim: 1024


In [20]:
# Check if all embeddings are normalized (unit norm)

from math import sqrt

test_embeddings_norm = [round(sqrt(sum(e**2)), 2) for e in test_embeddings]
sum(test_embeddings_norm) == len(test_embeddings)

True

In [21]:
# Embed all chunks
for i in tqdm(filttered_chunks, desc="Embedding chunks", unit="chunk"):
    i["embedding"] = embedding_model.encode(sentences=i["text"],
                                            dtype=torch.float32,
                                            convert_to_tensor=True)

random.sample(filttered_chunks, 1)[0]

Embedding chunks: 100%|██████████| 39/39 [00:03<00:00, 11.29chunk/s]


{'text': '●\u200b Ative notificações de movimentações na conta.\u200b  ●\u200b Mantenha o sistema do celular e o app atualizados.\u200b  ●\u200b Use senhas fortes e exclusivas.\u200b  Fraudes mais comuns e como evitar ●\u200b Phishing: e-mails ou mensagens falsas pedindo dados. Verifique sempre o remetente.\u200b  ●\u200b Links falsos: evite clicar em links não confiáveis.\u200b  ●\u200b Golpe do falso atendente: o banco nunca solicita senhas ou códigos por telefone.\u200b  ●\u200b Clonagem de WhatsApp: não compartilhe códigos recebidos por SMS.\u200b   12. Gerenciamento de Dados Pessoais Como alterar dados cadastrais\u200b Dados como telefone, e-mail, endereço e profissão podem ser atualizados no app em “Minha conta” > “Dados pessoais”. Alterações sensíveis, como nome e CPF, exigem envio de documentação. Exclusão de conta\u200b A exclusão pode ser solicitada no menu “Encerramento de conta” ou pelo atendimento. Após a solicitação, o banco inicia um processo que pode levar até 10 dias ú

## Embed storing

For simplicity, embeddings will be stored in a CSV file. However, when dealing with a large volume of embeddings (e.g., over 1M), a vector database (such as <a href="https://github.com/facebookresearch/faiss">FAISS</a>) becomes necessary.

In [22]:
from json import dumps # Using json to serialize the embeddings as strings
embeddings_df = pd.DataFrame(filttered_chunks)
embeddings_df["embedding"] = embeddings_df["embedding"].apply(lambda x: dumps(x.tolist()))
embeddings_df.to_csv(EMBEDDING_PATH, index=False)

# Retrieving & Generating

## Loading embeddings

In [23]:
import pandas as pd
from json import loads

embeddings_df = pd.read_csv(EMBEDDING_PATH)
embeddings_df["embedding"] = embeddings_df["embedding"].apply(lambda x: torch.tensor(loads(x), dtype=torch.float32))

In [24]:
embeddings_df.head(3)

,text,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,Manual de Atendimento e Uso – SafeBank 1. Int...,916,140,229.0,"[tensor(-0.0248), tensor(-0.0040), tensor(-0.0..."
1,Público-alvo O banco digital é voltado para p...,882,150,220.5,"[tensor(-0.0012), tensor(-0.0198), tensor(-0.0..."
2,Prazo para ativação da conta O processo de ve...,638,111,159.5,"[tensor(0.0321), tensor(-0.0184), tensor(-0.01..."


In [25]:
import numpy as np

embeddings = torch.tensor(data=np.array(embeddings_df["embedding"].tolist()),
                          dtype=torch.float32).to(DEVICE)
embeddings.shape

torch.Size([39, 1024])

## Query embedding

In [26]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path=EMBEDDING_MODEL,
                                      device=DEVICE,
                                      config_kwargs={"use_fp16": True})

In [27]:
query = "Como faço para abrir uma conta?"
query_embedding = embedding_model.encode(sentences=query,
                                         dtype=torch.float32,
                                         convert_to_tensor=True).to(DEVICE)
query_embedding

tensor([-0.0055, -0.0003, -0.0560,  ...,  0.0185, -0.0187, -0.0300],
       device='mps:0')

In [28]:
query_embedding.shape # same dimension as chunk embeddings

torch.Size([1024])

## Similarity search

### Dot product and Cosine similarity
Both are mathematical operations that take two vectors of the same length and return a single number (a scalar), which can be used to perform similarity search. The idea is that the larger this number is, the higher the chance of semantic similarity between the encoded texts.

### Dot product (GPT Text)
Takes into account both:
- How **aligned** (the angle) the vectors are (do they “point in a similar direction” → semantic similarity), and  
- How **large** the vectors are (magnitude), i.e., the length of each embedding. Larger magnitudes will increase the dot product.

### Cosine Similarity (GPT Text)
- Cares only about the **angle** (direction). It **ignores magnitude**. So two vectors pointing in exactly the same direction but with very different lengths will still have a cosine similarity close to 1, even if one vector’s magnitude is huge and the other small.

Mathematically, if two vectors (*v1*, *v2*) are normalized (which is our case), then:  
`dot_product(v1, v2) == cosine_similarity(v1, v2)`.  

So, to avoid unnecessary operations, if the vectors are already normalized (as in our embedding model), it is enough to perform the dot product to obtain the cosine similarity.

In general, it seems that the **direction** (rather than the magnitude) of the embedding vectors is the main factor in determining the semantics of the information. That’s why some embeddings already come normalized and why cosine similarity is generally preferred.

In [29]:
def get_similarity_scores(query_embedding: torch.Tensor, embeddings: torch.Tensor) -> torch.Tensor:
    """
    Compute similarity scores between the query embedding and all chunk embeddings
    using dot product since our embeddings are normalized (unit norm).
    """
    return torch.matmul(embeddings, query_embedding)

In [30]:
scores = get_similarity_scores(query_embedding, embeddings)
scores

tensor([0.5182, 0.6595, 0.5724, 0.5547, 0.5661, 0.5285, 0.5555, 0.5356, 0.4909,
        0.5140, 0.3968, 0.5009, 0.5293, 0.5166, 0.5018, 0.4516, 0.4690, 0.4944,
        0.5604, 0.5023, 0.4581, 0.5077, 0.5069, 0.5204, 0.5372, 0.5148, 0.5957,
        0.5560, 0.4126, 0.5071, 0.4969, 0.5871, 0.5324, 0.4860, 0.4538, 0.4345,
        0.4927, 0.4539, 0.4473], device='mps:0')

In [31]:
K = 3 # Number of top similar chunks to retrieve
top_k = torch.topk(scores, k=K)
top_k

torch.return_types.topk(
values=tensor([0.6595, 0.5957, 0.5871], device='mps:0'),
indices=tensor([ 1, 26, 31], device='mps:0'))

In [32]:
top_k_indices = top_k.indices.tolist()

relevant_chunks = embeddings_df.iloc[top_k_indices]["text"].tolist()
relevant_chunks

['Público-alvo  O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções financeiras modernas, acessíveis e com gestão 100% digital. 2. Cadastro e Abertura de Conta Requisitos para abertura de conta ●\u200b Ser maior de 18 anos ●\u200b Ter documento oficial com foto (RG, CNH ou RNE) ●\u200b Possuir um smartphone com câmera ●\u200b Endereço residencial  Etapas do processo de cadastro 1.\u200b Baixe o aplicativo ou acesse o site oficial 2.\u200b Informe dados pessoais (nome, CPF, e-mail, telefone) 3.\u200b Envie foto de documento oficial com foto 4.\u200b Grave um vídeo de reconhecimento facial 5.\u200b Confirme seu endereço e aceite os termos de uso Verificação de identidade (KYC)  O processo de KYC (Know Your Customer) é obrigatório para prevenir fraudes e garantir a segurança de todos os usuários. Após o envio dos documentos, nossa equipe faz a validação automática e manual dos dados.',
 '●\u200b Ative notificações de movimentações na conta.\u200b  ●\u20

## LLM

Running it locally!

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=LLM_MODEL_ID)
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=LLM_MODEL_ID,
                                                 dtype=torch.bfloat16, # keeping original precision
                                                 device_map=DEVICE)

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.97s/it]


In [34]:
llm_model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

In [48]:
print(f"LLM model vocab size: {tokenizer.vocab_size:,}")

LLM model vocab size: 256,000


In [36]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.
    """

    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Buffers are tensors that do not get updated during backpropagation. They act
    # like persistent, non-trainable states that you still want to keep track of.

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

print(f"Model num params: {get_model_num_params(llm_model):,}")

print(f"Model mem size: {get_model_mem_size(llm_model)["model_mem_gb"]}GB")

Model num params: 2,614,341,888
Model mem size: 4.87GB


In [37]:
test_message = [
    {"role": "user", "content": "Escreva um poema curto sobre o tema 'tecnologia e futuro', em português."}
]

# About chat templates: https://huggingface.co/docs/transformers/chat_templating
tokenized_test_message = tokenizer.apply_chat_template(conversation=test_message, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(DEVICE)

print(tokenizer.decode(tokenized_test_message[0]))

<bos><start_of_turn>user
Escreva um poema curto sobre o tema 'tecnologia e futuro', em português.<end_of_turn>
<start_of_turn>model



In [38]:
tokenized_test_message

tensor([[     2,    106,   1645,    108,   5305, 170503,   2378,  98787,  94762,
           4343,    493,  18952,    777,  16977,  65450,    526,  25817,    920,
           2190, 101619, 235265,    107,    108,    106,   2516,    108]],
       device='mps:0')

In [39]:
llm_output = llm_model.generate(tokenized_test_message, max_new_tokens=256)
print(llm_output)

print("")

llm_output_decoded = tokenizer.decode(llm_output[0])
print(llm_output_decoded)

tensor([[     2,    106,   1645,    108,   5305, 170503,   2378,  98787,  94762,
           4343,    493,  18952,    777,  16977,  65450,    526,  25817,    920,
           2190, 101619, 235265,    107,    108,    106,   2516,    108, 235280,
          23898,  59003,    527, 235269,   2378,  24177,    699,  51762, 235269,
            108,   1294,  25817, 235269,    476,  58974,    699,   1121,   8440,
         235265,    108,    958,    770,   5055,  97367,  41077, 235269,   2378,
          10000,    476,    699,  94347, 235269,    108,  10186,   9229,   5499,
         235269,   2378,  68905,    476,    699,    848,  32328,   7048, 235265,
            109,  10209, 181538,   2052, 140803, 235269, 110665,   2790,   9729,
            563, 235269,    108, 235280,  93244,  10417,  18225, 235269,   2378,
          22640,  31097,    480,   1351, 235265,    108, 235280,   8002,    699,
          54337, 235269,    476,  98519,    699,    582,    484,   1037,    524,
         235269,    108,  10

## Prompt augmenting

In [40]:
query = "Como faço para abrir uma conta?"
scores = get_similarity_scores(query_embedding, embeddings)
top_k_indices = torch.topk(scores, k=K).indices.tolist()
relevant_chunks = embeddings_df.iloc[top_k_indices]["text"].tolist()
relevant_chunks

['Público-alvo  O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções financeiras modernas, acessíveis e com gestão 100% digital. 2. Cadastro e Abertura de Conta Requisitos para abertura de conta ●\u200b Ser maior de 18 anos ●\u200b Ter documento oficial com foto (RG, CNH ou RNE) ●\u200b Possuir um smartphone com câmera ●\u200b Endereço residencial  Etapas do processo de cadastro 1.\u200b Baixe o aplicativo ou acesse o site oficial 2.\u200b Informe dados pessoais (nome, CPF, e-mail, telefone) 3.\u200b Envie foto de documento oficial com foto 4.\u200b Grave um vídeo de reconhecimento facial 5.\u200b Confirme seu endereço e aceite os termos de uso Verificação de identidade (KYC)  O processo de KYC (Know Your Customer) é obrigatório para prevenir fraudes e garantir a segurança de todos os usuários. Após o envio dos documentos, nossa equipe faz a validação automática e manual dos dados.',
 '●\u200b Ative notificações de movimentações na conta.\u200b  ●\u20

In [41]:
prompt = """
Baseado apenas no dado contexto, responda a pergunta que vem a seguir.
Contexto:
{context}
Pergunta: {question}
Resposta:
"""
formatted_prompt = [{"role": "user", "content": prompt.format(context="\n".join(relevant_chunks), question=query)}]
print(formatted_prompt)

[{'role': 'user', 'content': '\nBaseado apenas no dado contexto, responda a pergunta que vem a seguir.\nContexto:\nPúblico-alvo  O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções financeiras modernas, acessíveis e com gestão 100% digital. 2. Cadastro e Abertura de Conta Requisitos para abertura de conta ●\u200b Ser maior de 18 anos ●\u200b Ter documento oficial com foto (RG, CNH ou RNE) ●\u200b Possuir um smartphone com câmera ●\u200b Endereço residencial  Etapas do processo de cadastro 1.\u200b Baixe o aplicativo ou acesse o site oficial 2.\u200b Informe dados pessoais (nome, CPF, e-mail, telefone) 3.\u200b Envie foto de documento oficial com foto 4.\u200b Grave um vídeo de reconhecimento facial 5.\u200b Confirme seu endereço e aceite os termos de uso Verificação de identidade (KYC)  O processo de KYC (Know Your Customer) é obrigatório para prevenir fraudes e garantir a segurança de todos os usuários. Após o envio dos documentos, nossa equipe faz 

The final prompt plays a crucial role in obtaining high-quality responses from an LLM. Although many prompt-engineering techniques could be applied, the focus here was solely on the RAG pipeline itself.

Useful resources:
- <a href="https://www.promptingguide.ai/pt/introduction/examples">Prompting Guide</a>
- <a href="https://github.com/brexhq/prompt-engineering">Brex’s Prompt Engineering Guide</a>
- <a href="https://arxiv.org/abs/2401.14423">Related arXiv Paper</a>


In [42]:
tokenized_prompt = tokenizer.apply_chat_template(conversation=formatted_prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(DEVICE)
tokenized_prompt.shape

torch.Size([1, 554])

In [43]:
llm_output = llm_model.generate(tokenized_prompt, max_new_tokens=256)
llm_output.shape

torch.Size([1, 642])

In [44]:
decoded_llm_output = tokenizer.decode(llm_output[0])
print(decoded_llm_output)

<bos><start_of_turn>user
Baseado apenas no dado contexto, responda a pergunta que vem a seguir.
Contexto:
Público-alvo  O banco digital é voltado para pessoas físicas a partir de 18 anos, que buscam soluções financeiras modernas, acessíveis e com gestão 100% digital. 2. Cadastro e Abertura de Conta Requisitos para abertura de conta ●​ Ser maior de 18 anos ●​ Ter documento oficial com foto (RG, CNH ou RNE) ●​ Possuir um smartphone com câmera ●​ Endereço residencial  Etapas do processo de cadastro 1.​ Baixe o aplicativo ou acesse o site oficial 2.​ Informe dados pessoais (nome, CPF, e-mail, telefone) 3.​ Envie foto de documento oficial com foto 4.​ Grave um vídeo de reconhecimento facial 5.​ Confirme seu endereço e aceite os termos de uso Verificação de identidade (KYC)  O processo de KYC (Know Your Customer) é obrigatório para prevenir fraudes e garantir a segurança de todos os usuários. Após o envio dos documentos, nossa equipe faz a validação automática e manual dos dados.
●​ Ative no

In [45]:
filtered_llm_response = decoded_llm_output.split("<start_of_turn>model")[1].replace("<end_of_turn>", "").strip()
print(filtered_llm_response)

Para abrir uma conta no banco digital, siga estes passos:

1. **Baixe o aplicativo ou acesse o site oficial.**
2. **Informe dados pessoais (nome, CPF, e-mail, telefone).**
3. **Envie foto de documento oficial com foto.**
4. **Grave um vídeo de reconhecimento facial.**
5. **Confirme seu endereço e aceite os termos de uso.**
